## Prepare data

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
train = pd.read_csv('train_small.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.dropna(subset=['type'], inplace=True)

In [4]:
test = pd.read_csv('labeled_output.csv')

In [5]:
test.dropna(subset=['text'], inplace=True)

In [6]:
test['category']=np.nan

In [7]:
X = train['text']
y = train['category']

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=32, test_size=0.2)

In [9]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)

In [10]:
import keras
from sklearn.preprocessing import LabelEncoder

num_classes=6
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
y_train.shape
y_train

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.]], dtype=float32)

## Neural Network

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vec = TfidfVectorizer(analyzer='word',
                         min_df=1,
                         strip_accents='unicode',
                         token_pattern=r'\w{1,}',
                         ngram_range=(2,3),
                         )
x_train_vec = tf_vec.fit_transform(x_train)
x_test_vec = tf_vec.transform(x_test)
print (x_train_vec.shape)
print (x_test_vec.shape)

(207, 2163)
(52, 2163)


In [13]:
import keras
from keras import models
from keras import layers

In [14]:
input_tensor = layers.Input(shape=(2163,), name='input')
hidden_layer_1 = layers.Dense(units=16, activation='relu', name='hidden_layer_1')(input_tensor)
hidden_layer_2 = layers.Dense(units=16, activation='relu', name='hidden_layer_2')(hidden_layer_1)
output_tensor = layers.Dense(6, activation='softmax', name='output')(hidden_layer_2)

In [15]:
network = models.Model(inputs=input_tensor, outputs=output_tensor)

network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 2163)              0         
_________________________________________________________________
hidden_layer_1 (Dense)       (None, 16)                34624     
_________________________________________________________________
hidden_layer_2 (Dense)       (None, 16)                272       
_________________________________________________________________
output (Dense)               (None, 6)                 102       
Total params: 34,998
Trainable params: 34,998
Non-trainable params: 0
_________________________________________________________________


In [16]:
from keras import optimizers
network.compile(optimizer=optimizers.RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
network.fit(x=x_train_vec, y=y_train, batch_size=128, epochs=40)

Epoch 1/40
207/207 [==============================] - 0s 1ms/step - loss: 1.7913 - acc: 0.3671
Epoch 2/40
207/207 [==============================] - 0s 68us/step - loss: 1.7725 - acc: 0.5990
Epoch 3/40
207/207 [==============================] - 0s 70us/step - loss: 1.7575 - acc: 0.6329
Epoch 4/40
207/207 [==============================] - 0s 68us/step - loss: 1.7438 - acc: 0.6184
Epoch 5/40
207/207 [==============================] - 0s 54us/step - loss: 1.7303 - acc: 0.6425
Epoch 6/40
207/207 [==============================] - 0s 65us/step - loss: 1.7170 - acc: 0.6473
Epoch 7/40
207/207 [==============================] - 0s 66us/step - loss: 1.7039 - acc: 0.6425
Epoch 8/40
207/207 [==============================] - 0s 66us/step - loss: 1.6907 - acc: 0.6570
Epoch 9/40
207/207 [==============================] - 0s 67us/step - loss: 1.6774 - acc: 0.6618
Epoch 10/40
207/207 [==============================] - 0s 66us/step - loss: 1.6635 - acc: 0.6570
Epoch 11/40
207/207 [===================